In [ ]:
from paddleocr import PaddleOCR, draw_ocr
from matplotlib import pyplot as plt
import cv2
import os
import ollama
import json

In [ ]:
# Setup model
ocr_model = PaddleOCR(lang='en', use_gpu=False)


[2025/02/08 18:17:00] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\Akshat/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\Akshat/.paddleocr/whl\\rec\\en\\en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_nu

In [ ]:
def get_strings_simple(data):
    """Extract strings using simple list comprehension"""
    return [item[1] for item in data[0]]

[2025/02/08 18:17:01] ppocr WARNING: Since the angle classifier is not initialized, it will not be used during the forward process
[2025/02/08 18:17:02] ppocr DEBUG: dt_boxes num : 36, elapsed : 0.13854718208312988
[2025/02/08 18:17:04] ppocr DEBUG: rec_res num  : 36, elapsed : 2.2875618934631348


In [ ]:
def query_qwen2(text):
    """Sends text to the locally running Qwen2.5-3B model and forces valid JSON output."""
    prompt = f"""
    Extract structured details from the following receipt text. Respond only in JSON format without any extra text.
    **Input Receipt Text:**
    \"\"\"{text}\"\"\"
    **JSON Format (Example Output):**
    {{
        "vendor": "Walmart",
        "amount": "45.67",
        "date": "2024-02-08",
        "category": "Grocery"
    }}
    Now, extract details from the given receipt and return only JSON:
    """
    response = ollama.chat(model='qwen2.5:3b', messages=[{"role": "user", "content": prompt}])
    json_text = response['message']['content'].strip()
    # Ensure only valid JSON is returned
    try:
        structured_data = json.loads(json_text)  # Parse JSON response
        return structured_data
    except json.JSONDecodeError:
        print("Error: Model response is not valid JSON.\nResponse:\n", json_text)
        return None

In [ ]:
def process_receipt(ocr_text_list):
    """Takes OCR-extracted text and processes it using Qwen2.5-3B to extract structured receipt details."""
    combined_text = " ".join([text[0] for text in ocr_text_list])  # Combine OCR-extracted text
    structured_data = query_qwen2(combined_text)  # Send to Qwen model
    return structured_data

In [ ]:
# Read and process image
result = ocr_model.ocr('images/0.jpg')

In [ ]:
# Extract strings
strings = get_strings_simple(result)

In [ ]:
# Process receipt
receipt_details = process_receipt(strings)
print("Extracted Receipt Details:", receipt_details)